In [1]:
import pandas as pd, sqlite3, requests
import hockey_scraper as hs
import numpy as np
from datetime import date, timedelta, datetime

In [2]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
pbpDates = pd.read_sql("SELECT MAX(Date) FROM PBP WHERE Date < '2017-07-01'", conn)
maxDate = pbpDates.iloc[0]["MAX(Date)"]
maxDate

'2017-02-28'

In [2]:
#variables globales

conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
fin = " WHERE EVENT = 'GOAL' AND Period <> '5' AND PBP.Date >= ALIGNEMENTS.dateDebut AND PBP.Date <= ALIGNEMENTS.dateFin AND PBP.Date >= '2021-01-13' GROUP BY idAlignement)) WHERE idAlignement = ALIGNEMENTS.idAlignement)"

#scrape les points
def scrapePoints():
    ok = "Scrape fait"
    pbpDates = pd.read_sql("SELECT MAX(Date) FROM PBP", conn)
    maxDate = pbpDates.iloc[0]["MAX(Date)"]
    maxDate = pd.to_datetime(maxDate) + timedelta(days=1)
    maxDate = maxDate.strftime("%Y-%m-%d")
    ajd = date.today()
    ajd = ajd.strftime("%Y-%m-%d")

    try:
        scrape = hs.nhl.scrape_functions.scrape_date_range(maxDate, ajd,
                                if_scrape_shifts=True, data_format='pandas', preseason=False, rescrape=False, docs_dir=True)
        
        #pbp
        dfPBP = scrape["pbp"]
        dfPBP.to_sql('PBP', conn, if_exists='append', index = False)

        #shifts
        dfshifts = scrape["shifts"]
        dfshifts.to_sql('SHIFTS', conn, if_exists='append', index = False)
    except:
        print("PBP à jour")
    return ok

#fonctions
def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

def writeUpdateDebut(typeScore):
    updateDebut = "UPDATE ALIGNEMENTS SET " + typeScore + "Actuels = (SELECT Nb" + typeScore + " FROM ((SELECT idAlignement, COUNT(Event) AS Nb" + typeScore + " FROM PBP INNER JOIN ALIGNEMENTS ON"
    return updateDebut
    
def majInterne(typeScore):
    debut = writeUpdateDebut(typeScore)
    if typeScore == "buts":
        milieu = " (PBP.p1_ID = ALIGNEMENTS.idNHL)"
    elif typeScore == "assist":
        milieu = " (PBP.p2_ID = ALIGNEMENTS.idNHL) OR (PBP.p3_ID = ALIGNEMENTS.idNHL)"
    sqlUpdate = debut + milieu + fin
    execSQL(sqlUpdate, conn, write = True)

def writeSetZero(typeScore):
    setZero = "UPDATE ALIGNEMENTS SET " + typeScore + "Actuels = 0 WHERE " + typeScore + "Actuels IS NULL"
    return setZero

def majPointsAlignements():
    setButsZero = writeSetZero("buts")
    setAssistZero = writeSetZero("assist")
    setPtsZero = writeSetZero("points")
    sqlUpdate = "UPDATE ALIGNEMENTS SET pointsActuels = (butsActuels + assistActuels)"
    execSQL(setButsZero, conn, write = True)
    execSQL(setAssistZero, conn, write = True)
    execSQL(setPtsZero, conn, write = True)
    execSQL(sqlUpdate, conn, write = True)

def majPJ():
    sqlUpdate = "UPDATE ALIGNEMENTS SET pjActuels = (SELECT NbPJ FROM (SELECT idAlignement, COUNT(DISTINCT Game_Id) AS NbPJ FROM SHIFTS INNER JOIN ALIGNEMENTS ON (SHIFTS.Player_Id = ALIGNEMENTS.idNHL) AND SHIFTS.Date >= ALIGNEMENTS.dateDebut AND SHIFTS.Date <= ALIGNEMENTS.dateFin AND SHIFTS.Date >= '2021-01-13' GROUP BY idAlignement) WHERE idAlignement = ALIGNEMENTS.idAlignement)"
    updateNull = "UPDATE ALIGNEMENTS SET pjActuels = 0 WHERE pjActuels IS NULL"
    execSQL(sqlUpdate, conn, write = True)
    execSQL(updateNull, conn, write = True)

def majPJBPPAlign():
    majInterne("buts")
    majInterne("assist")
    majPointsAlignements()
    majPJ()

def majScorePoolers():
    sqlUpdate = "UPDATE POOLERS SET PJ = (SELECT NbPJ FROM (SELECT idPooler, SUM(pjActuels) AS NbPJ FROM ALIGNEMENTS WHERE statutJoueur = 'Alignement' GROUP BY idPooler) WHERE Id = idPooler), B = (SELECT NbB FROM (SELECT idPooler, SUM(butsActuels) AS NbB FROM ALIGNEMENTS WHERE statutJoueur = 'Alignement' GROUP BY idPooler) WHERE Id = idPooler), A = (SELECT NbA FROM (SELECT idPooler, SUM(assistActuels) AS NbA FROM ALIGNEMENTS WHERE statutJoueur = 'Alignement' GROUP BY idPooler) WHERE Id = idPooler), Score = (SELECT NbSc FROM (SELECT idPooler, SUM(pointsActuels) AS NbSc FROM ALIGNEMENTS WHERE statutJoueur = 'Alignement' GROUP BY idPooler) WHERE Id = idPooler)"
    execSQL(sqlUpdate, conn, write = True)
    
#maj du score des gardiens
def logGardien(idNHL):
    jsonJoueur = requests.get("https://statsapi.web.nhl.com/api/v1/people/" + idNHL + "/stats?stats=gameLog&season=20202021").json()
    df = pd.json_normalize(jsonJoueur["stats"][0]["splits"])
    gardienLogs = df[["season", "date", "isWin", "stat.shutouts"]]
    gardienLogs["gardien"] = idNHL
    return gardienLogs

def GwriteUpdate(typeScore):
    colAl = ""
    if typeScore == "isWin":
        colAl = "buts"
    elif typeScore == "'stat.shutouts'":
        colAl = "assist"
    elif typeScore == "scorePool":
        colAl = "points"
    debut = "UPDATE ALIGNEMENTS SET " + colAl + "Actuels = (SELECT Nb FROM (SELECT gardien, SUM(" + typeScore + ") AS Nb FROM GARDIENS WHERE dateMatch >= ALIGNEMENTS.dateDebut AND dateMatch <= ALIGNEMENTS.dateFin AND gardien = ALIGNEMENTS.idNHL)) WHERE ALIGNEMENTS.idNHL IN (SELECT DISTINCT(gardien) FROM GARDIENS)"
    return debut

def majScoreGardiens():
    dfGardiens = pd.read_sql("SELECT idNHL FROM JOUEURS WHERE position == 'G'", conn)
    gardiens = dfGardiens["idNHL"].apply(str)
    listGardiens = gardiens.values.tolist()

    vraisCol = ["season", "date", "isWin", "stat.shutouts", "gardien"]
    gameLogsG = pd.DataFrame(columns=vraisCol)
    gameLogsG = gameLogsG.fillna(0)

    for i in listGardiens:
        try:
            gardienLogs = logGardien(i)
        except:
            print("fausse alarme (espoir) " + i)
        gameLogsG = gameLogsG.append(gardienLogs)
    gameLogsG = gameLogsG.rename(columns = {"date" : "dateMatch"})
    gameLogsG[["dateMatch"]] = gameLogsG.dateMatch.str.split(" 00:",expand=True)
    #gameLogsG["dateMatch"] = pd.to_datetime(gameLogsG["dateMatch"])
    pbpDates = pd.read_sql("SELECT MAX(Date) FROM PBP", conn)
    maxDate = pbpDates.iloc[0]["MAX(Date)"]
    gameLogsG = gameLogsG[(gameLogsG["dateMatch"] <= maxDate)]
    
    a = gameLogsG["isWin"]
    gameLogsG["esWin"] = a * 2
    gameLogsG ["scorePool"] = gameLogsG["esWin"] + gameLogsG["stat.shutouts"]
    gameLogsG.to_sql("GARDIENS", conn, if_exists="replace", index=False)

    updVic = GwriteUpdate("isWin")
    updJB = GwriteUpdate("'stat.shutouts'")
    updSco = GwriteUpdate("scorePool")
    execSQL(updVic, conn, write = True)
    execSQL(updJB, conn, write = True)
    execSQL(updSco, conn, write = True)

#majMasseSalariale()
def majMasseSalariale():
    sqlUpdate = "UPDATE POOLERS SET MasseSalariale = (SELECT masseActuelle FROM (SELECT idPooler, SUM(salaireActuel) AS masseActuelle FROM ALIGNEMENTS INNER JOIN JOUEURS ON ALIGNEMENTS.idNHL = JOUEURS.idNHL WHERE date('now') BETWEEN ALIGNEMENTS.dateDebut AND ALIGNEMENTS.dateFin AND (statutJoueur = 'Alignement' OR statutJoueur = 'Réserve') GROUP BY idPooler) WHERE Poolers.Id = idPooler)"
    execSQL(sqlUpdate, conn, write = True)

#maj des json
def majTables():
    conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
    var = 'OK'
    try:
        #maj Poolers
        dfPoolers = pd.read_sql("SELECT * FROM POOLERS", conn)
        jsonPoolers = dfPoolers.to_json("C:/dev/pool-de-hockey/src/data/poolers.json", orient = "table", index =    False, indent = 4)
    except Exception as ex:
        print("Erreur : poolers")
        var = "Erreur poolers"

    try:
        #maj Alignements
        requete = "SELECT * FROM ALIGNEMENTS INNER JOIN JOUEURS ON JOUEURS.idNHL = ALIGNEMENTS.idNHL"
        dfTempo = pd.read_sql(requete, conn)
        dfAlignements = dfTempo.T.drop_duplicates().T
        jsonAlignements = dfAlignements.to_json("C:/dev/pool-de-hockey/src/data/alignements.json", orient = "table", index = False, indent = 4)
    except Exception as ex2:
        print("Erreur : alignements")
        var = "Erreur alignements"

    try:
        #maj Joueurs
        dfJoueurs = pd.read_sql("SELECT * FROM JOUEURS", conn)
        jsonJoueurs = dfJoueurs.to_json("C:/dev/pool-de-hockey/src/data/joueurs.json", orient = "table", index = False, indent = 4)
    except Exception as ex3:
        print("Erreur : joueurs")
        var = "Erreur joueurs"

    return var


def scrape_dates(date_inf, date_sup):
    ok = "Scrape fait"
    test = pd.date_range(date_inf,date_sup)
    pbpDates = pd.read_sql("SELECT DISTINCT(Date) FROM PBP", conn)
    dates = pbpDates["Date"]
    result = []
    for i in dates:
        if i in test:
            result.append(1)
        else:
            result.append(0)
    if sum(result) > 0:
        return "Erreur: tentative de scraper des dates qui sont déjà dans SQLite."
    else:
        try:
            scrape = hs.nhl.scrape_functions.scrape_date_range(date_inf, date_sup,
                                    if_scrape_shifts=True, data_format='pandas', preseason=False, rescrape=False, docs_dir=True)

            #pbp
            dfPBP = scrape["pbp"]
            dfPBP.to_sql('PBP', conn, if_exists='append', index = False)

            #shifts
            dfshifts = scrape["shifts"]
            dfshifts.to_sql('SHIFTS', conn, if_exists='append', index = False)
        except:
            print("PBP à jour")
        return ok


In [9]:
#scrapePoints()
print("1")
#majPJBPPAlign()
print("2")
#majScoreGardiens()
print("3")
#majScorePoolers()
print("4")
majMasseSalariale()
print("5")
majTables()
print("6")

1
2
3
4
5
6


In [9]:
#avant de commencer le scraping, il faut voir si j'ai des matchs en trop
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
df_test = pd.read_sql("SELECT Game_Id, Date, Event, Away_Team, Home_Team FROM PBP WHERE Event IS 'GEND'", conn)
team = "OTT"
df_test["is_AMTL"] = np.where(df_test["Away_Team"] == team,
                    1, 0)
df_test["is_HMTL"] = np.where(df_test["Home_Team"] == team,
                    1, 0)
df_test["is_MTL"] = df_test["is_AMTL"] + df_test["is_HMTL"]                    
df_test = df_test[df_test["is_MTL"] == 1]
df_test[['Annee','Mois','Jour']] = df_test['Date'].str.split('-',expand=True)
df_test["Annee"] = pd.to_numeric(df_test["Annee"])
df_test["Mois"] = pd.to_numeric(df_test["Mois"])
df_test["saison_series"] =  df_test['Game_Id'].astype(str).str[0]
df_test = df_test[df_test["saison_series"] == "2"]
df_test["Saison"] = np.where(df_test["Mois"] > 8,
                    df_test["Annee"], df_test["Annee"] - 1)
df_test["Saison"] = df_test["Saison"].map(str)
df_test["Game_Id"] = df_test["Saison"] + df_test["Game_Id"]
df_agg = df_test.groupby(['Saison']).count()
df2020 = df_test[df_test["Saison"] == "2020"]
## Tout semble correct!
df_agg
#note: df_agg permet de voir le nb de matchs par saison d'une équipe dans la db.

,Game_Id,Date,Event,Away_Team,Home_Team,is_AMTL,is_HMTL,is_MTL,Annee,Mois,Jour,saison_series
Saison,,,,,,,,,,,,
2017,81,81,81,81,81,81,81,81,81,81,81,81
2018,82,82,82,82,82,82,82,82,82,82,82,82
2019,71,71,71,71,71,71,71,71,71,71,71,71
2020,55,55,55,55,55,55,55,55,55,55,55,55


In [90]:
pbpDates = pd.read_sql("SELECT MAX(Date) FROM PBP", conn)
maxDate = pbpDates.iloc[0]["MAX(Date)"]
#maxDate = maxDate.strftime("%Y-%m-%d")
maxDate

'2021-02-28'

In [3]:
#scraper la saison 2015-2016
#la saison commence le 7 octobre 2015
# et finit le 10 avril 2016

#rendu au 1er janvier 2016
scrape_dates("2016-01-01", "2016-04-10")

Scraping Game  2015020565 2016-01-01
Scraping Game  2015020566 2016-01-01
Scraping Game  2015020567 2016-01-02
Scraping Game  2015020569 2016-01-02
Scraping Game  2015020568 2016-01-02
Scraping Game  2015020570 2016-01-02
Scraping Game  2015020571 2016-01-02
Scraping Game  2015020572 2016-01-02
Scraping Game  2015020573 2016-01-02
Scraping Game  2015020574 2016-01-02
Scraping Game  2015020575 2016-01-02
Scraping Game  2015020576 2016-01-02
Scraping Game  2015020578 2016-01-02
Scraping Game  2015020577 2016-01-02
Scraping Game  2015020579 2016-01-03
Scraping Game  2015020580 2016-01-03
Scraping Game  2015020581 2016-01-03
Scraping Game  2015020582 2016-01-03
Scraping Game  2015020583 2016-01-04
Scraping Game  2015020584 2016-01-04
Scraping Game  2015020585 2016-01-04
Scraping Game  2015020586 2016-01-04
Scraping Game  2015020587 2016-01-04
Scraping Game  2015020588 2016-01-05
Scraping Game  2015020589 2016-01-05
Scraping Game  2015020590 2016-01-05
Scraping Game  2015020592 2016-01-05
S

Scraping Game  2015020591 2016-01-05
Scraping Game  2015020594 2016-01-05
Scraping Game  2015020595 2016-01-05
Scraping Game  2015020596 2016-01-06
Scraping Game  2015020597 2016-01-06
Scraping Game  2015020599 2016-01-06
Scraping Game  2015020600 2016-01-06
Scraping Game  2015020598 2016-01-06
Scraping Game  2015020601 2016-01-07
Scraping Game  2015020602 2016-01-07
Scraping Game  2015020603 2016-01-07
Scraping Game  2015020604 2016-01-07
Scraping Game  2015020605 2016-01-07
Scraping Game  2015020606 2016-01-07
Scraping Game  2015020607 2016-01-07
Scraping Game  2015020608 2016-01-08
Scraping Game  2015020609 2016-01-08
Scraping Game  2015020610 2016-01-08
Scraping Game  2015020611 2016-01-08
Scraping Game  2015020612 2016-01-08
Scraping Game  2015020613 2016-01-08
Scraping Game  2015020614 2016-01-09
Scraping Game  2015020615 2016-01-09
Scraping Game  2015020619 2016-01-09
Scraping Game  2015020616 2016-01-09
Scraping Game  2015020617 2016-01-09
Scraping Game  2015020618 2016-01-09
S

Scraping Game  2015020996 2016-03-08
Scraping Game  2015020994 2016-03-08
Scraping Game  2015020997 2016-03-08
Scraping Game  2015020998 2016-03-08
Scraping Game  2015020999 2016-03-08
Scraping Game  2015021000 2016-03-08
Scraping Game  2015021001 2016-03-09
Scraping Game  2015021002 2016-03-09
Scraping Game  2015021003 2016-03-09
Scraping Game  2015021005 2016-03-09
Scraping Game  2015021004 2016-03-09
Scraping Game  2015021006 2016-03-09
Scraping Game  2015021007 2016-03-10
Scraping Game  2015021008 2016-03-10
Scraping Game  2015021009 2016-03-10
Scraping Game  2015021010 2016-03-10
Scraping Game  2015021011 2016-03-10
Scraping Game  2015021012 2016-03-10
Scraping Game  2015021013 2016-03-11
Scraping Game  2015021014 2016-03-11
Scraping Game  2015021015 2016-03-11
Scraping Game  2015021016 2016-03-11
Scraping Game  2015021017 2016-03-11
Scraping Game  2015021018 2016-03-12
Scraping Game  2015021020 2016-03-12
Scraping Game  2015021019 2016-03-12
Scraping Game  2015021024 2016-03-12
S

'Scrape fait'